In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/predict-volcanic-eruptions-ingv-oe/train.csv")

Adapted the Preproccessing method from this great [notebook](https://www.kaggle.com/isaienkov/ingv-volcanic-eruption-prediction-eda-modeling) by Kostiantyn Isaienkov.

In [ ]:
def build_row(values, segment_id, sensor_number):
    row = pd.DataFrame()
    row.loc[segment_id, f'{sensor_number}_sum'] = values.sum()
    row.loc[segment_id, f'{sensor_number}_mean'] = values.mean()
    row.loc[segment_id, f'{sensor_number}_min'] = values.min()
    row.loc[segment_id, f'{sensor_number}_max'] = values.max()
    row.loc[segment_id, f'{sensor_number}_kurtosis'] = values.kurtosis()
    row.loc[segment_id, f'{sensor_number}_var'] = values.var()
    row.loc[segment_id, f'{sensor_number}_mad'] = values.mad()
    row.loc[segment_id, f'{sensor_number}_skew'] = values.skew()
    row.loc[segment_id, f'{sensor_number}_q99'] = np.quantile(values, 0.99)
    row.loc[segment_id, f'{sensor_number}_q01'] = np.quantile(values, 0.01)
    row.loc[segment_id, f'{sensor_number}_q25'] = np.quantile(values, 0.25)
    row.loc[segment_id, f'{sensor_number}_q75'] = np.quantile(values, 0.75)
    row.loc[segment_id, f'{sensor_number}_q95'] = np.quantile(values, 0.95)
    row.loc[segment_id, f'{sensor_number}_q05'] = np.quantile(values, 0.05)
    row.loc[segment_id, f'{sensor_number}_q10'] = np.quantile(values, 0.1)
    row.loc[segment_id, f'{sensor_number}_q90'] = np.quantile(values, 0.9)
    row.loc[segment_id, f'{sensor_number}_q85'] = np.quantile(values, 0.85)
    row.loc[segment_id, f'{sensor_number}_q15'] = np.quantile(values, 0.15)
    row.loc[segment_id, f'{sensor_number}_q80'] = np.quantile(values, 0.8)
    row.loc[segment_id, f'{sensor_number}_q20'] = np.quantile(values, 0.2)
    row.loc[segment_id, f'{sensor_number}_q60'] = np.quantile(values, 0.6)
    row.loc[segment_id, f'{sensor_number}_q40'] = np.quantile(values, 0.4)
    
    return row

In [ ]:
train_set = list()
j=0
for seg in train.segment_id:
    signals = pd.read_csv(f'/kaggle/input/predict-volcanic-eruptions-ingv-oe/train/{seg}.csv')
    train_row = []
    if j%500 == 0:
        print(j)
    for i in range(0, 10):
        sensor_id = f'sensor_{i+1}'
        train_row.append(build_row(signals[sensor_id].fillna(0), seg, sensor_id))
    train_row = pd.concat(train_row, axis=1)
    train_set.append(train_row)
    j+=1

train_set = pd.concat(train_set)

In [ ]:
train_set = train_set.reset_index()
train_set = train_set.rename(columns={'index': 'segment_id'})
train_set = pd.merge(train_set, train, on='segment_id')
train_set

In [ ]:
train_set.to_csv('train_set.csv', index=False)

In [ ]:
test = pd.read_csv("/kaggle/input/predict-volcanic-eruptions-ingv-oe/sample_submission.csv")

In [ ]:
test_set = list()
j=0
for seg in test.segment_id:
    signals = pd.read_csv(f'/kaggle/input/predict-volcanic-eruptions-ingv-oe/test/{seg}.csv')
    test_row = []
    if j%50 == 0:
        print(j)
    for i in range(0, 10):
        sensor_id = f'sensor_{i+1}'
        test_row.append(build_row(signals[sensor_id].fillna(0), seg, sensor_id))
    test_row = pd.concat(test_row, axis=1)
    test_set.append(test_row)
    j+=1

test_set = pd.concat(test_set)

In [ ]:
test_set = test_set.reset_index()
test_set = test_set.rename(columns={'index': 'segment_id'})
test_set = pd.merge(test_set, test, on='segment_id')
test_set

In [ ]:
test_set.to_csv('test_set.csv', index=False)